In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [44]:
import itertools
import numpy as np
from scipy.optimize import linprog
from cvxopt import matrix, solvers, spmatrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from initialize import linparams

In [3]:
# Parameter values
n_class = 2
los = 3
capacity = 50
intensity = 0.9
slope_init = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

In [4]:
# intercepts and slopes for rate class, arrival day of week and los combination
params = linparams(capacity, intensity, slope_init, rates_init)

In [5]:
# Starting parameters
params_slopes = params[0]
params_intercepts = params[1]

In [6]:
params_slopes

array([[[-0.08122807, -0.07370269, -0.08375278],
        [-0.07544926, -0.08451997, -0.08142716],
        [-0.09322178, -0.07725568, -0.05773602],
        [-0.1068316 , -0.07412914, -0.06613756],
        [-0.08873431, -0.08650088, -0.07492901],
        [-0.09686144, -0.07815779, -0.06201683],
        [-0.0760522 , -0.07928701, -0.06180266]],

       [[-0.15483036, -0.13341666, -0.11434043],
        [-0.12637321, -0.09987329, -0.11079515],
        [-0.15404708, -0.11475417, -0.09260816],
        [-0.18590262, -0.09726223, -0.10104369],
        [-0.192081  , -0.13001256, -0.09647752],
        [-0.16013092, -0.11018703, -0.12105246],
        [-0.17224569, -0.08826453, -0.08778424]]])

In [7]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [8]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [27]:
obj_coefs = (-1) * rates_arrival_los.reshape(n_class * 7 * los)
type(obj_coefs)

numpy.ndarray

In [72]:
# Inequality equations, LHS
# We have total number of 42 decision veriables, corresponding to total number of
# rate class, arrival day of week and los combination.
C_ineq = np.zeros(n_class * 7 * los * n_class * 7).reshape(n_class*7, n_class*7*los)
C_ineq[0,:(7*los)] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
C_ineq[1,:(7*los)] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
C_ineq[2,:(7*los)] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[3,:(7*los)] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[4,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
C_ineq[5,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
C_ineq[6,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
C_ineq[7,(7*los):] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
C_ineq[8,(7*los):] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
C_ineq[9,(7*los):] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[10,(7*los):] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
C_ineq[11,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
C_ineq[12,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
C_ineq[13,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]

# Inequality equations, RHS
# For each rate class, number of arrivals for a stay night in question is half of
# expected demand, which is capacity * intensity
cap_ineq = capacity * intensity * 0.5 * np.ones(n_class * 7)


In [73]:
D_ineq = np.identity(42, dtype=float)
A_ineq = np.concatenate((D_ineq, C_ineq), axis=0)

In [75]:
A_ineq.shape

(56, 42)

In [15]:
A_ineq[:7, :21], A_ineq[7:, 21:]

(array([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 1., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         1., 1., 1., 1., 1.]]),
 array([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 1., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.

In [32]:
type(A_ineq)

numpy.ndarray

In [16]:
res = linprog(obj_coefs, A_ub=A_ineq, b_ub=b_ineq)

In [17]:
bkLimits = np.round(res['x'], decimals=0)

In [18]:
bkLimits

array([12.,  2.,  2., 12.,  2.,  2., 12.,  2.,  2., 12.,  2.,  2., 12.,
        2.,  2., 12.,  2.,  2., 12.,  2.,  2., 14.,  1.,  1., 13.,  2.,
        3., 13.,  2.,  1., 13.,  1.,  2., 14.,  2.,  2., 12.,  2.,  2.,
       12.,  2.,  2.])

In [19]:
print(res)

     con: array([], dtype=float64)
     fun: -37124.999999791864
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([1.25748301e-10, 1.27315047e-10, 1.27947430e-10, 1.27521105e-10,
       1.25929489e-10, 1.25826460e-10, 1.25830013e-10, 1.25758959e-10,
       1.26107125e-10, 1.25215394e-10, 1.25030652e-10, 1.24622090e-10,
       1.26313182e-10, 1.26341604e-10])
  status: 0
 success: True
       x: array([12.37158357,  2.1423201 ,  1.79342164, 12.07762305,  2.32009402,
        2.18204824, 12.09092037,  2.32009403,  1.79342171, 12.07762313,
        2.14231999,  1.9844929 , 12.37158367,  2.00772268,  2.20045905,
       11.95337328,  2.15349315,  2.20045894, 11.95337311,  2.00772279,
        1.98449296, 13.96220902,  1.27128166,  0.85236837, 13.06841819,
        2.44723225,  2.77822914, 13.12256962,  2.44723225,  0.85236837,
       13.06841819,  1.27128166,  2.08247039, 13.96220902,  1.89536753,
        2.43630303, 11.7325492 ,  1.91700681,  2.43630303, 11.7325492

In [40]:
b = matrix(b_ineq)
A = matrix(A_ineq)
c = matrix(obj_coefs, tc='d')

In [43]:
type(A_ineq)

numpy.ndarray

In [56]:
sol = solvers.lp(c, A, b)

ValueError: Rank(A) < p or Rank([G; A]) < n

In [65]:
spindex = np.where(A_ineq == 1)

In [66]:
A = spmatrix(1.0, spindex[0], spindex[1])

In [68]:
print(A), print(c), print(b)

[ 1.00e+00  1.00e+00  1.00e+00     0         0         0         0     ... ]
[    0      1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00     0     ... ]
[    0         0      1.00e+00     0      1.00e+00  1.00e+00  1.00e+00 ... ]
[    0         0         0         0         0      1.00e+00     0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0         0         0         0     ... ]

(None, None, None)

In [69]:
sol = solvers.lp(c, A, b)

ValueError: Rank(A) < p or Rank([G; A]) < n

In [61]:
c_test = matrix([-4., -5.])
G_test = matrix([[2., 1., -1., 0.], [1., 2., 0., -1.]])
h_test = matrix([3., 3., 0., 0.])
sol_test = solvers.lp(c_test, G_test, h_test)

     pcost       dcost       gap    pres   dres   k/t
 0: -8.1000e+00 -1.8300e+01  4e+00  0e+00  8e-01  1e+00
 1: -8.8055e+00 -9.4357e+00  2e-01  2e-16  4e-02  3e-02
 2: -8.9981e+00 -9.0049e+00  2e-03  5e-16  5e-04  4e-04
 3: -9.0000e+00 -9.0000e+00  2e-05  2e-16  5e-06  4e-06
 4: -9.0000e+00 -9.0000e+00  2e-07  1e-16  5e-08  4e-08
Optimal solution found.


In [62]:
print(c_test)

[-4.00e+00]
[-5.00e+00]



In [63]:
print(G_test)

[ 2.00e+00  1.00e+00]
[ 1.00e+00  2.00e+00]
[-1.00e+00  0.00e+00]
[ 0.00e+00 -1.00e+00]



In [64]:
print(h_test)

[ 3.00e+00]
[ 3.00e+00]
[ 0.00e+00]
[ 0.00e+00]

